In [1]:
import os
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from typing import Tuple

In [2]:
def encode(df: pd.DataFrame) -> Tuple[pd.DataFrame, LabelEncoder, LabelEncoder]:
    userId_label_encoder = LabelEncoder()
    movieId_label_encoder = LabelEncoder()

    df['userId'] = userId_label_encoder.fit_transform(df['userId'].values)
    df['movieId'] = movieId_label_encoder.fit_transform(df['movieId'].values)

    # encoder.inverse_transform() 으로 decode
    return df, userId_label_encoder, movieId_label_encoder

In [3]:
class DirFilePath:
    dir_base = os.path.join(os.path.join('/opt','ml','paper','RecSys'))
    dir_data = os.path.join(dir_base, 'Data', 'ml-latest-small')
    path_rating = os.path.join(dir_data, 'ratings.csv')

In [5]:
df_rating = pd.read_csv(DirFilePath.path_rating)

df_rating, user_encoder, movie_encoder = encode(df_rating)

In [6]:
df_rating

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,2,4.0,964981247
2,0,5,4.0,964982224
3,0,43,5.0,964983815
4,0,46,5.0,964982931
...,...,...,...,...
100831,609,9416,4.0,1493848402
100832,609,9443,5.0,1493850091
100833,609,9444,5.0,1494273047
100834,609,9445,5.0,1493846352


In [7]:
set_users = set(df_rating['userId'].unique())
num_users = len(set_users)
set_movies = set(df_rating['movieId'].unique())
num_movies = len(set_movies)
user_sequence_movies = [
    df_rating[df_rating['userId']==user].sort_values(by='timestamp', axis=0)['movieId'].tolist()
    for user in set_users
]
use_negative_movies = 